# Yellow Taxi Data query in Vast DB - Part 3

In [4]:
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [5]:
%load_ext sql

In [6]:
%sql trino://admin@${DOCKER_HOST_OR_IP}:8443/vast  --connection_arguments '{"http_scheme":"https", "verify": false}'

Connecting to 'trino://admin@10.143.11.241:8443/vast'

In [7]:
%%sql

show schemas in vast

Running query in 'trino://admin@10.143.11.241:8443/vast'

Schema
csnow-db|social_media
csnow-db|taxi
csnow-db|taxi_data
information_schema
system_schema
vast-audit-log-bucket|vast_audit_log_schema
vast-big-catalog-bucket|vast_big_catalog_schema


In [15]:
import os
VASTDB_TWITTER_INGEST_BUCKET = os.getenv('VASTDB_TWITTER_INGEST_BUCKET')
print(VASTDB_TWITTER_INGEST_BUCKET)

csnow-db


In [21]:
%%sql
    
show tables in vast."{{VASTDB_TWITTER_INGEST_BUCKET}}|taxi_data"

Running query in 'trino://admin@10.143.11.241:8443/vast'

Table
yellow_tripdata


In [22]:
%%sql

select 
    format('%,d', COUNT(*)) as row_count 
from
    vast."{{VASTDB_TWITTER_INGEST_BUCKET}}|taxi_data".yellow_tripdata

Running query in 'trino://admin@10.143.11.241:8443/vast'

row_count
"1,763,456,499"


In [23]:
%%sql

SELECT
    *
FROM
    vast."{{VASTDB_TWITTER_INGEST_BUCKET}}|taxi_data".yellow_tripdata
LIMIT 10

Running query in 'trino://admin@10.143.11.241:8443/vast'

airport_fee,congestion_surcharge,dolocationid,dropoff_latitude,dropoff_longitude,extra,fare_amount,improvement_surcharge,mta_tax,passenger_count,payment_type,pulocationid,pickup_latitude,pickup_longitude,ratecodeid,store_and_fwd_flag,surcharge,tip_amount,tolls_amount,total_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,vendorid
-999.0,-999.0,-999,40.753277,-73.971878,-999.0,6.5,-999.0,None,4,Cash,-999,40.766916,-73.959782,None,0.0,0.0,0.0,0.0,6.5,2009-08-25 21:21:58,2009-08-25 21:13:52,1.3,CMT
-999.0,-999.0,-999,40.747361,-74.003237,-999.0,11.3,-999.0,None,4,Cash,-999,40.720521,-74.010067,None,0.0,0.0,0.0,0.0,11.3,2009-08-18 23:03:44,2009-08-18 22:48:30,3.2,CMT
-999.0,-999.0,-999,40.747285,-73.985802,-999.0,8.099999999999998,-999.0,None,1,Cash,-999,40.723361,-74.003314,None,0.0,0.0,0.0,0.0,8.099999999999998,2009-08-18 22:58:43,2009-08-18 22:48:38,2.2,CMT
-999.0,-999.0,-999,40.756299,-73.990949,-999.0,4.5,-999.0,None,1,Cash,-999,40.748928,-73.986525,None,0.0,0.0,0.0,0.0,4.5,2009-08-17 19:38:12,2009-08-17 19:34:45,0.6999999999999998,CMT
-999.0,-999.0,-999,40.787437,-73.97392999999998,-999.0,18.1,-999.0,None,3,Credit,-999,40.734751,-73.990651,None,0.0,0.0,2.86,0.0,20.96,2009-08-18 16:35:19,2009-08-18 16:06:27,4.9,CMT
-999.0,-999.0,-999,40.687696,-74.18147899999998,-999.0,80.0,-999.0,None,1,Credit,-999,40.687687,-74.181572,None,0.0,0.0,13.5,10.0,103.5,2009-08-17 16:08:32,2009-08-17 16:07:45,0.1,CMT
-999.0,-999.0,-999,40.739471,-73.987505,-999.0,10.9,-999.0,None,5,Cash,-999,40.758939,-73.985855,None,0.0,0.0,0.0,0.0,10.9,2009-08-18 19:08:53,2009-08-18 18:51:03,2.3,CMT
-999.0,-999.0,-999,40.774174,-73.986964,-999.0,9.7,-999.0,None,1,Cash,-999,40.759507,-73.96799699999998,None,0.0,0.0,0.0,0.0,9.7,2009-08-19 00:09:29,2009-08-18 23:54:51,2.5,CMT
-999.0,-999.0,-999,40.774258,-73.951604,-999.0,18.5,-999.0,None,1,Credit,-999,40.70989,-74.01056,None,0.0,0.0,1.0,0.0,19.5,2009-08-07 11:54:05,2009-08-07 11:36:45,7.0,CMT
-999.0,-999.0,-999,40.756489,-73.99006099999998,-999.0,7.7,-999.0,None,1,Cash,-999,40.764507,-73.973983,None,0.0,0.0,0.0,0.0,7.7,2009-08-17 16:15:49,2009-08-17 16:03:48,1.4,CMT


In [25]:
%%sql

SELECT
    DISTINCT vendorid
FROM
    vast."{{VASTDB_TWITTER_INGEST_BUCKET}}|taxi_data".yellow_tripdata
LIMIT 10

Running query in 'trino://admin@10.143.11.241:8443/vast'

HttpError: error 404: b'404 page not found\n'